## Comparing compiled vs interpreted language performance.

#### Ziad Arafat - 2023-03-05

The performance of compiled and interpreted languages has been a topic of interest in programming languages research for decades. While compiled languages are generally known for their faster runtime performance, interpreted languages offer advantages in terms of ease of use and portability. In this experiment, we aimed to explore the practical performance of compiled and interpreted languages in implementing Gaussian elimination with back substitution, a widely used numerical algorithm. We implemented the algorithm in Python and FORTRAN, and tested their runtime performance on matrices of varying size, with and without the use of numpy. The experiment provides insights into the practical trade-offs between compiled and interpreted languages in the context of numerical algorithms.

#### These commands will clean and compile our fortran program so its ready to execute

In [ ]:
!pip install -U pip

In [ ]:
!make clean
!make

### Python function to generate a random matrix of size nxn

In [ ]:
import random

def generate_matrix_and_vector(n):
        """
        This function generates a random square matrix of size n x n.
        
        Args:
        n: the size of the square matrix
        
        Returns:
        a list of lists representing the random square matrix
        
        """
        min_value = -2000.0
        max_value = 2000.0
        
        matrix = []

        for row in range(n):
                next_row = []
                for column in range(n):
                        next_row.append(random.uniform(min_value, max_value))
                matrix.append(next_row)
        vector = [random.uniform(min_value, max_value) for number in range(n)]
        
        return matrix, vector

### Python function to measure our runtime

In [ ]:
import time

def measure_time(func, *args, **kwargs):
    """
    This function measures the time it takes to run another function with a given set of arguments.
    
    Args:
    func: the function to be measured
    *args: the positional arguments for the function
    **kwargs: the keyword arguments for the function
    
    Returns:
    the runtime of the function in seconds
    
    """
    start_time = time.time()
    func(*args, **kwargs)
    end_time = time.time()
    runtime = end_time - start_time
    return runtime


### Python without numpy

In [ ]:
def gaussian_elimination(matrix, vector):
    """
    This function performs Gaussian elimination with back substitution on a given square matrix A and vector b, 
    without partial pivoting or using numpy.
    
    Args:
    A: a square matrix of size n x n
    b: a vector of length n
    
    Returns:
    x: a vector of length n that represents the solution to the linear system Ax = b
    
    """
    n = len(matrix)
    
    # Forward elimination
    for i in range(n):
        # Check if the diagonal element is zero
        if matrix[i][i] == 0:
            raise ValueError('Diagonal element is zero, cannot proceed with Gaussian elimination')
        # Normalize the current row
        for j in range(i + 1, n):
            ratio = matrix[j][i] / matrix[i][i]
            for k in range(i, n):
                matrix[j][k] -= ratio * matrix[i][k]
            vector[j] -= ratio * vector[i]
    
    # Back substitution
    x = [0] * n
    for i in range(n - 1, -1, -1):
        x[i] = vector[i]
        for j in range(i + 1, n):
            x[i] -= matrix[i][j] * x[j]
        x[i] /= matrix[i][i]
    
    return x


### Python with numpy

In [ ]:
import numpy as np
import scipy

def gaussian_elimination_lu(matrix, vector):
    """
    This function performs Gaussian elimination with back substitution using LU decomposition with partial pivoting.
    
    Args:
    n: the size of the square matrix
    matrix: the square matrix as a list of lists
    vector: the vector as a list
    
    Returns:
    the solution vector as a NumPy array
    
    """
    # Convert the matrix and vector to NumPy arrays
    

    # Perform LU decomposition with partial pivoting
    pivoted, L, U = scipy.linalg.lu(matrix)

    # Solve the system using back substitution
    y = np.linalg.solve(L, pivoted.dot(vector))
    x = np.linalg.solve(U, y)

    return x


### Python functions to run our fortran program

#### Function to generate the command line args

In [ ]:
def generate_fortran_args(matrix, vector):
        """
        This function generates command line arguments for a Fortran program.

        Args:
        matrix: the square matrix as a list of lists
        vector: the vector as a list

        Returns:
        a list of command line arguments

        """
        # Create a new matrix with the vector appended as a new column
        new_matrix = [row + [vector[index]] for index, row in enumerate(matrix)]
        n = len(matrix)
        
        matrix_string = " ".join([str(x) for row in new_matrix for x in row])
        with open("matrix.txt", "w") as file:
                file.write(matrix_string)
        # Flatten the matrix into a list of arguments and start with dimensions
         
        args = [str(n), str(n + 1)] + ["matrix.txt"]

        return args


#### Function to execute the fortran command

In [ ]:
import subprocess

def execute_fortran_program(args):
    """
    This function executes a local Fortran program with given command line arguments.
    
    Args:
    args: a list of command line arguments
    
    Returns:
    the output of the Fortran program
    
    """
    
    # Run the Fortran program with the given arguments using subprocess
#     print(args)
    output = subprocess.check_output(['./gaussian_elimination'] + args, universal_newlines=True)
    
    return output


#### Function to execute the fortran with the generated args and measure the runtime

In [ ]:
def run_and_measure_fortran():
    args = generate_fortran_args(10,
                                 generate_matrix_and_vector(10),
                                 [random.randint(0, 100) for _ in range(10)])
    print(args)
    execute_fortran_program(args)
    


sample_size = 5
input_sizes = (250,)
algorithms = ("fortran", "python no numpy", "python with numpy")
results = dict(zip(algorithms, 
              [dict(zip(input_sizes, 
                   [[] for item in input_sizes])) for _ in algorithms]))
matrix, vector = generate_matrix_and_vector(5)
print(matrix)
matrix_np = np.array(matrix, dtype=float)
vector_np = np.array(vector, dtype=float)
print(matrix_np.shape)
for n_size in input_sizes:
        for sample in range(sample_size):
                matrix, vector = generate_matrix_and_vector(n_size)
                fortran_args = generate_fortran_args(matrix, vector)
                print("There are",len(fortran_args), "ARGS")
                
                print("Executing Fortran with", n_size, "number", sample+1)
                result_fortran = measure_time(execute_fortran_program,
                                              fortran_args)
                results["fortran"][n_size].append(result_fortran)
                # execute_fortran_program(fortran_args)
                
                print("Executing python with", n_size, "number", sample+1)
                result_python = measure_time(gaussian_elimination, 
                                             matrix, vector)
                results["python no numpy"][n_size].append(result_python)
                
                print("Executing numpy with", n_size, "number", sample+1)
                matrix_np = np.array(matrix, dtype=float)
                vector_np = np.array(vector, dtype=float)
                print(matrix_np.shape, vector_np.shape)
                result_python_numpy = measure_time(gaussian_elimination_lu,
                                             matrix_np, vector_np)
                results["python with numpy"][n_size].append(result_python_numpy)    

In [ ]:
print(results)